In [10]:
from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import cv2
import os


image_types = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".pgm")


def list_images(basePath, contains=None):
    # return the set of files that are valid
    return list_files(basePath, validExts=image_types, contains=contains)


def list_files(basePath, validExts=None, contains=None):
    # loop over the directory structure
    for (rootDir, dirNames, filenames) in os.walk(basePath):
        # loop over the filenames in the current directory
        for filename in filenames:
            # if the contains string is not none and the filename does not contain
            # the supplied string, then ignore the file
            if contains is not None and filename.find(contains) == -1:
                continue

            # determine the file extension of the current file
            ext = filename[filename.rfind("."):].lower()

            # check to see if the file is an image and should be processed
            if validExts is None or ext.endswith(validExts):
                # construct the path to the image and yield it
                imagePath = os.path.join(rootDir, filename)
                yield imagePath
                
args = dict()
args['dataset'] = 'dataset'
args['embeddings'] = 'output/embeddings.pickle'
args['detector'] = 'face_detection_model'
args['embedding_model'] = 'openface_nn4.small2.v1.t7'
args['confidence'] = 0.5

# load our serialized face detector from disk
print("[INFO] loading face detector...")
protoPath = os.path.sep.join([args["detector"], "deploy.prototxt"])
modelPath = os.path.sep.join([args["detector"],
    "res10_300x300_ssd_iter_140000.caffemodel"])
detector = cv2.dnn.readNetFromCaffe(protoPath, modelPath)

# load our serialized face embedding model from disk
print("[INFO] loading face recognizer...")
embedder = cv2.dnn.readNetFromTorch(args["embedding_model"])

# grab the paths to the input images in our dataset
print("[INFO] quantifying faces...")
imagePaths = list(list_images(args["dataset"]))

# initialize our lists of extracted facial embeddings and
# corresponding people names
knownEmbeddings = []
knownNames = []

# initialize the total number of faces processed
total = 0

# print(imagePaths)

def get_emotion(name):
    emo = ['sad','happy','angry','neutral']
    for e in emo:
        if e in name:
            return e

# loop over the image paths
for (i, imagePath) in enumerate(imagePaths):
    # extract the person name from the image path
    print("[INFO] processing image {}/{}".format(i + 1,
        len(imagePaths)))
    name = imagePath.split(os.path.sep)[-1]
    
    
    name = get_emotion(name)

    # load the image, resize it to have a width of 600 pixels (while
    # maintaining the aspect ratio), and then grab the image
    # dimensions
    image = cv2.imread(imagePath)
    image = imutils.resize(image, width=600)
    (h, w) = image.shape[:2]

    # construct a blob from the image
    imageBlob = cv2.dnn.blobFromImage(
        cv2.resize(image, (300, 300)), 1.0, (300, 300),
        (104.0, 177.0, 123.0), swapRB=False, crop=False)

    # apply OpenCV's deep learning-based face detector to localize
    # faces in the input image
    detector.setInput(imageBlob)
    detections = detector.forward()

    # ensure at least one face was found
    if len(detections) > 0:
        # we're making the assumption that each image has only ONE
        # face, so find the bounding box with the largest probability
        i = np.argmax(detections[0, 0, :, 2])
        confidence = detections[0, 0, i, 2]

        # ensure that the detection with the largest probability also
        # means our minimum probability test (thus helping filter out
        # weak detections)
        if confidence > args["confidence"]:
            # compute the (x, y)-coordinates of the bounding box for
            # the face
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # extract the face ROI and grab the ROI dimensions
            face = image[startY:endY, startX:endX]
            (fH, fW) = face.shape[:2]

            # ensure the face width and height are sufficiently large
            if fW < 20 or fH < 20:
                continue

            # construct a blob for the face ROI, then pass the blob
            # through our face embedding model to obtain the 128-d
            # quantification of the face
            faceBlob = cv2.dnn.blobFromImage(face, 1.0 / 255,
                (96, 96), (0, 0, 0), swapRB=True, crop=False)
            embedder.setInput(faceBlob)
            vec = embedder.forward()

            # add the name of the person + corresponding face
            # embedding to their respective lists
            knownNames.append(name)
            knownEmbeddings.append(vec.flatten())
            total += 1

# dump the facial embeddings + names to disk
print("[INFO] serializing {} encodings...".format(total))
data = {"embeddings": knownEmbeddings, "names": knownNames}
f = open(args["embeddings"], "wb")
f.write(pickle.dumps(data))
f.close()

[INFO] loading face detector...
[INFO] loading face recognizer...
[INFO] quantifying faces...
[INFO] processing image 1/1872
[INFO] processing image 2/1872
[INFO] processing image 3/1872
[INFO] processing image 4/1872
[INFO] processing image 5/1872
[INFO] processing image 6/1872
[INFO] processing image 7/1872
[INFO] processing image 8/1872
[INFO] processing image 9/1872
[INFO] processing image 10/1872
[INFO] processing image 11/1872
[INFO] processing image 12/1872
[INFO] processing image 13/1872
[INFO] processing image 14/1872
[INFO] processing image 15/1872
[INFO] processing image 16/1872
[INFO] processing image 17/1872
[INFO] processing image 18/1872
[INFO] processing image 19/1872
[INFO] processing image 20/1872
[INFO] processing image 21/1872
[INFO] processing image 22/1872
[INFO] processing image 23/1872
[INFO] processing image 24/1872
[INFO] processing image 25/1872
[INFO] processing image 26/1872
[INFO] processing image 27/1872
[INFO] processing image 28/1872
[INFO] processing i

[INFO] processing image 255/1872
[INFO] processing image 256/1872
[INFO] processing image 257/1872
[INFO] processing image 258/1872
[INFO] processing image 259/1872
[INFO] processing image 260/1872
[INFO] processing image 261/1872
[INFO] processing image 262/1872
[INFO] processing image 263/1872
[INFO] processing image 264/1872
[INFO] processing image 265/1872
[INFO] processing image 266/1872
[INFO] processing image 267/1872
[INFO] processing image 268/1872
[INFO] processing image 269/1872
[INFO] processing image 270/1872
[INFO] processing image 271/1872
[INFO] processing image 272/1872
[INFO] processing image 273/1872
[INFO] processing image 274/1872
[INFO] processing image 275/1872
[INFO] processing image 276/1872
[INFO] processing image 277/1872
[INFO] processing image 278/1872
[INFO] processing image 279/1872
[INFO] processing image 280/1872
[INFO] processing image 281/1872
[INFO] processing image 282/1872
[INFO] processing image 283/1872
[INFO] processing image 284/1872
[INFO] pro

[INFO] processing image 512/1872
[INFO] processing image 513/1872
[INFO] processing image 514/1872
[INFO] processing image 515/1872
[INFO] processing image 516/1872
[INFO] processing image 517/1872
[INFO] processing image 518/1872
[INFO] processing image 519/1872
[INFO] processing image 520/1872
[INFO] processing image 521/1872
[INFO] processing image 522/1872
[INFO] processing image 523/1872
[INFO] processing image 524/1872
[INFO] processing image 525/1872
[INFO] processing image 526/1872
[INFO] processing image 527/1872
[INFO] processing image 528/1872
[INFO] processing image 529/1872
[INFO] processing image 530/1872
[INFO] processing image 531/1872
[INFO] processing image 532/1872
[INFO] processing image 533/1872
[INFO] processing image 534/1872
[INFO] processing image 535/1872
[INFO] processing image 536/1872
[INFO] processing image 537/1872
[INFO] processing image 538/1872
[INFO] processing image 539/1872
[INFO] processing image 540/1872
[INFO] processing image 541/1872
[INFO] pro

[INFO] processing image 762/1872
[INFO] processing image 763/1872
[INFO] processing image 764/1872
[INFO] processing image 765/1872
[INFO] processing image 766/1872
[INFO] processing image 767/1872
[INFO] processing image 768/1872
[INFO] processing image 769/1872
[INFO] processing image 770/1872
[INFO] processing image 771/1872
[INFO] processing image 772/1872
[INFO] processing image 773/1872
[INFO] processing image 774/1872
[INFO] processing image 775/1872
[INFO] processing image 776/1872
[INFO] processing image 777/1872
[INFO] processing image 778/1872
[INFO] processing image 779/1872
[INFO] processing image 780/1872
[INFO] processing image 781/1872
[INFO] processing image 782/1872
[INFO] processing image 783/1872
[INFO] processing image 784/1872
[INFO] processing image 785/1872
[INFO] processing image 786/1872
[INFO] processing image 787/1872
[INFO] processing image 788/1872
[INFO] processing image 789/1872
[INFO] processing image 790/1872
[INFO] processing image 791/1872
[INFO] pro

[INFO] processing image 1016/1872
[INFO] processing image 1017/1872
[INFO] processing image 1018/1872
[INFO] processing image 1019/1872
[INFO] processing image 1020/1872
[INFO] processing image 1021/1872
[INFO] processing image 1022/1872
[INFO] processing image 1023/1872
[INFO] processing image 1024/1872
[INFO] processing image 1025/1872
[INFO] processing image 1026/1872
[INFO] processing image 1027/1872
[INFO] processing image 1028/1872
[INFO] processing image 1029/1872
[INFO] processing image 1030/1872
[INFO] processing image 1031/1872
[INFO] processing image 1032/1872
[INFO] processing image 1033/1872
[INFO] processing image 1034/1872
[INFO] processing image 1035/1872
[INFO] processing image 1036/1872
[INFO] processing image 1037/1872
[INFO] processing image 1038/1872
[INFO] processing image 1039/1872
[INFO] processing image 1040/1872
[INFO] processing image 1041/1872
[INFO] processing image 1042/1872
[INFO] processing image 1043/1872
[INFO] processing image 1044/1872
[INFO] process

[INFO] processing image 1265/1872
[INFO] processing image 1266/1872
[INFO] processing image 1267/1872
[INFO] processing image 1268/1872
[INFO] processing image 1269/1872
[INFO] processing image 1270/1872
[INFO] processing image 1271/1872
[INFO] processing image 1272/1872
[INFO] processing image 1273/1872
[INFO] processing image 1274/1872
[INFO] processing image 1275/1872
[INFO] processing image 1276/1872
[INFO] processing image 1277/1872
[INFO] processing image 1278/1872
[INFO] processing image 1279/1872
[INFO] processing image 1280/1872
[INFO] processing image 1281/1872
[INFO] processing image 1282/1872
[INFO] processing image 1283/1872
[INFO] processing image 1284/1872
[INFO] processing image 1285/1872
[INFO] processing image 1286/1872
[INFO] processing image 1287/1872
[INFO] processing image 1288/1872
[INFO] processing image 1289/1872
[INFO] processing image 1290/1872
[INFO] processing image 1291/1872
[INFO] processing image 1292/1872
[INFO] processing image 1293/1872
[INFO] process

[INFO] processing image 1512/1872
[INFO] processing image 1513/1872
[INFO] processing image 1514/1872
[INFO] processing image 1515/1872
[INFO] processing image 1516/1872
[INFO] processing image 1517/1872
[INFO] processing image 1518/1872
[INFO] processing image 1519/1872
[INFO] processing image 1520/1872
[INFO] processing image 1521/1872
[INFO] processing image 1522/1872
[INFO] processing image 1523/1872
[INFO] processing image 1524/1872
[INFO] processing image 1525/1872
[INFO] processing image 1526/1872
[INFO] processing image 1527/1872
[INFO] processing image 1528/1872
[INFO] processing image 1529/1872
[INFO] processing image 1530/1872
[INFO] processing image 1531/1872
[INFO] processing image 1532/1872
[INFO] processing image 1533/1872
[INFO] processing image 1534/1872
[INFO] processing image 1535/1872
[INFO] processing image 1536/1872
[INFO] processing image 1537/1872
[INFO] processing image 1538/1872
[INFO] processing image 1539/1872
[INFO] processing image 1540/1872
[INFO] process

[INFO] processing image 1761/1872
[INFO] processing image 1762/1872
[INFO] processing image 1763/1872
[INFO] processing image 1764/1872
[INFO] processing image 1765/1872
[INFO] processing image 1766/1872
[INFO] processing image 1767/1872
[INFO] processing image 1768/1872
[INFO] processing image 1769/1872
[INFO] processing image 1770/1872
[INFO] processing image 1771/1872
[INFO] processing image 1772/1872
[INFO] processing image 1773/1872
[INFO] processing image 1774/1872
[INFO] processing image 1775/1872
[INFO] processing image 1776/1872
[INFO] processing image 1777/1872
[INFO] processing image 1778/1872
[INFO] processing image 1779/1872
[INFO] processing image 1780/1872
[INFO] processing image 1781/1872
[INFO] processing image 1782/1872
[INFO] processing image 1783/1872
[INFO] processing image 1784/1872
[INFO] processing image 1785/1872
[INFO] processing image 1786/1872
[INFO] processing image 1787/1872
[INFO] processing image 1788/1872
[INFO] processing image 1789/1872
[INFO] process